In [1]:
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random
import torch.utils.data as Data
# pytorch neural network
import torch.nn as nn


# 首先生成简单人工数据集
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

# 读取数据集，每次随机读取 batch_size 大小的数据集
batch_size = 10
# 将训练数据集的 特征 & 标签 组合
dataset = Data.TensorDataset(features, labels)
# 随机读取 小批量数据
# data_iter 和 此前我们手写的 data_iter函数效果一样
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)


# 定义模型
pytorch提供大量预定义的层，我们只需关注使用哪些层来构造模型，下面将介绍如何使用PyTorch更简洁定义 线性回归。<br><br>
首先，导入`torch.nn`模块。实际上，“nn”是neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的层。之前我们已经用过了`autograd`，而`nn`就是利用`autograd`来定义模型。`nn`的核心数据结构是`Module`，它是一个抽象概念，既可以表示神经网络中的某个**层**（layer），也可以表示一个包含很**多层的神经网络**。在实际使用中，最常见的做法是继承`nn.Module`，撰写自己的网络层。一个`nn.Module`实例应该包含一些层以及返回输出的前向传播（forward）方法。下面先来看看如何用`nn.Module`实现一个线性回归模型。

In [ ]:
class LinearNet(nn.Module):
    def __init__(self, n_features):
        super(LinearNet, self).__init__()
        # 线性层，输入n_features 个特征，输出1个特征
        # 这里的 self.linear == 之前我们手写的 linreg
        self.linear = nn.Linear(n_features, 1)
        # 后面添加 weight 和 bias
        self.weight = self.linear.weight
        self.bias = self.linear.bias
        # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

In [ ]:
# 完全自定义模型
class LinearNetMy(nn.Module):
    def __init__(self, n_input, n_output):
        super(LinearNetMy, self).__init__()
        self.weight = nn.Parameter(torch.randn(n_input, n_output))
        # self.bias = nn.Parameter(torch.randn(n_output))
        self.bias = nn.Parameter(torch.randn(1, n_output))
    def forward(self, x):
        y = torch.matmul(x, self.weight) + self.bias
        return y

事实上，我们还可以用 `nn.Sequential` 更加方便搭建网络，`Sequential`是一个有序容器，网络层将按照 传入 `Sequential` 的顺序以此被添加到计算图中

In [ ]:
# 写法1
net = nn.Sequential(
    # 线性层，输入 num_inputs 个特征，输出 1 个特征
    nn.Linear(num_inputs, 1)
    # 此处还可以添加更多 网络层
)

# 写法2
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ...

# 写法3
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
    ('linear', nn.Linear(num_inputs, 1))
    # ......
]))

print(net)
print(net[0])

可以通过 `net.parameters()` 查看模型所有的可学习 参数，此函数将返回一个生成器
线性层 其实就是一个单层神经网络，而且输入特征 与 输出特征 全连接，线性层又叫 全连接层

In [ ]:
for param in net.parameters():
    print(param)

# 初始化模型参数
在使用`net`前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。PyTorch在`init`模块中提供了多种参数初始化方法。这里的init是initializer的缩写形式。我们通过`init.normal_`将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零。<br>
> 注：如果这里的`net`是通过继承nn.Module类自定义的，那么上面代码会报错，`net[0].weight`应改为`net.linear.weight`，`bias`亦然。因为`net[0]`这样根据下标访问子模块的写法只有当`net`是个`ModuleList`或者`Sequential`实例时才可以。

In [ ]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0) # 也可以直接修改 bias 的 data: net[0].bias.data.fill_(0)

# 定义损失函数
PyTorch在`nn`模块中提供了各种损失函数，这些损失函数可看作是一种特殊的层，PyTorch也将这些损失函数实现为nn.Module的子类。我们现在使用它提供的均方误差损失作为模型的损失函数。

In [6]:
loss = nn.MSELoss() # 该 loss 已经求和

# 定义优化算法
同样，我们也无须自己实现小批量随机梯度下降算法。`torch.optim`模块提供了很多常用的优化算法比如`SGD`、`Adam`和`RMSProp`等。下面我们创建一个用于优化`net`所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

我们还可以为不同 网络层 设置不同的学习率，这在finetune时经常用到
```python
optimizer = optim.SGD([
    # 如果对某个参数不指定学习率，就使用最外层的默认学习率
    {'params': net.subnet1.parameters()}, # lr = 0.03
    {'params': net.subnet2.parameters(), 'lr': 0.01}
], lr=0.03)
```
有时候我们不想让学习率固定成一个常数，那如何调整学习率呢？主要有两种做法。一种是修改optimizer.param_groups中对应的学习率，另一种是更简单也是较为推荐的做法——新建优化器，由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer。但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况。

In [8]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

# 模型训练

In [ ]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于 net.zero_grad()
        l.backward() # 向参数 累计梯度
        optimizer.step() # 根据当前梯度 优化
    print('epoch %d, loss: %f' % (epoch, l.item()))

# 下面对最终迭代出来的 参数进行评估
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)
